In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors
from tqdm import tqdm_notebook 

# sklearn is only used for making the train test split

from sklearn.model_selection import train_test_split

In [ ]:
my_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","yellow","green"])
np.random.seed(9)# for using recreatability
df = pd.read_csv('data1.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().sum() 

In [ ]:
X = df[['variance',' skewness','curtosis','entropy']].to_numpy()
Y = df['class'].to_numpy()

In [ ]:
df['class'].value_counts()

In [ ]:
# normalization 
# using  mean and varience so that the weights don't blow up
X_norm = np.zeros_like(X,dtype=float)
for i in range(X[0,:].size):
    X_norm[:,i] = (X[:,i] - X[:,i].mean()) / X[:,i].std()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_norm, Y,stratify=Y ,test_size=0.2, random_state=0)

In [ ]:
class SigmoidNeuron:
  
  def __init__(self,reg = "none",loss_f = 'none',lam = 1):
    self.w = None
    self.b = 0
    self.reg = reg
    self.lam = lam
    self.loss_f = loss_f
    self.loss_tracing = {}
    self.accuracy_tracking = {}
    
  def perceptron(self, x):
    return np.dot(x, self.w.T) + self.b
  
  def sigmoid(self, x):
    return 1.0/(1.0 + np.exp(-x))
  
  def grad_w(self, x, y):
        y_pred = self.sigmoid(self.perceptron(x))
        
        return (y_pred - y) * y_pred * (1 - y_pred) * x

  def pos_neg(self):
          w_=np.zeros_like(self.w)
          w = self.w.tolist()
          
          for j in range(len(w)):
              
              if w[0][j]>0:
                  w_[j] = 1
              else:
                  w_[j] = -1
 
          return np.array(w_)
  def grad_b(self, x, y):
    y_pred = self.sigmoid(self.perceptron(x))
    return (y_pred - y) * y_pred * (1 - y_pred)

  def accuracy_calc(self,X,Y):
      y_pred = self.sigmoid(self.perceptron(X))
      Y_bin = 1*(.8 <= y_pred)
      
      accuracy_cal = (Y_bin==Y.all()).sum()
      return accuracy_cal/Y.size

  def loss_cal(self,pred,actual):
        
        loss = 0
       
        for y,y_p in zip(pred,actual):
            loss += (y-y_p)**2
        
        if self.loss_f == 'none':
            return (loss**.5)/pred.size
        elif self.loss_f == 'l2':
            return ((loss**.5)+np.dot(self.w,self.w.T).item())/pred.size+np.dot(self.w,self.w.T).item()
        elif self.loss_f == 'l1' :
            return ((loss**.5)+np.absolute(self.w).sum())/pred.size
  
  def fit(self, X, Y, epochs=1, learning_rate=.1, initialise='normal', display_loss=False):
    
    # initialise w, b
    if initialise == 'random':
        self.w = np.random.rand(1, X.shape[1]) #N * 4 will create random with (1,4)
        self.b = 0
    elif initialise == 'normal':
        self.w = np.random.randn(1,X.shape[1]) #N * 4 will create random with (1,4) but all number bell mean =0 std=1
        self.b=0
      
    m = Y.size
    for i in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
      dw = 0 #change
      db = 0 # change b
for x, y in zip(X, Y):#x will be each of X(one row of X) y = corespoding row of Y
        dw += self.grad_w(x, y)# for all points
        db += self.grad_b(x, y) 
      if self.reg == 'l2':      
        self.w -=   (learning_rate * self.lam) * self.w + learning_rate * (dw)
        self.b -=  learning_rate*db
      elif self.reg == 'l1':
            self.w -= (learning_rate * self.lam) * self.pos_neg() + dw * learning_rate
            self.b -=  learning_rate*db
      else :
            self.w -= learning_rate*dw
            self.b -= learning_rate*db
      
      if display_loss:
        Y_pred = self.sigmoid(self.perceptron(X))
        self.loss_tracing[i] = self.loss_cal(Y_pred, Y)
        self.accuracy_tracking[i] = self.accuracy_calc(X_test,Y_test)
    print("Accuracy=",self.accuracy_calc(X_train,Y_train))
    if display_loss:
        fig, (ax1, ax2) = plt.subplots(2,sharex=True)
        fig.suptitle('Loss verses Test Acuracy')
        ax1.plot(list(self.loss_tracing.values()))
        ax1.set_title("loss v/s epoches")
        ax2.plot(list(self.accuracy_tracking.values()))
        ax2.set_title("accuracy v/s epoches")
      
      
        plt.show()
      
  def predict(self, X):
    Y_pred = []
    for x in X:
      y_pred = self.sigmoid(self.perceptron(x))
      Y_pred.append(y_pred)
    return np.array(Y_pred)

In [ ]:
sn1 = SigmoidNeuron()
sn2 = SigmoidNeuron(reg='l2',lam = 5)
sn3 = SigmoidNeuron(reg='l1',lam = 5)

In [ ]:
sn1.fit(X_train,Y_train,epochs=100,learning_rate=0.015,display_loss=False)
sn2.fit(X_train,Y_train,epochs=100,learning_rate=0.015,display_loss=False)
sn3.fit(X_train,Y_train,epochs=100,learning_rate=0.015,display_loss=False)